In [ ]:
!pip3 install pyquery
#1 CRAWLING
from bs4 import BeautifulSoup
import requests
import pyquery
import json
import time
import sys
data = []
link_abstraks = []
data_abstrak = []
##ambil semua link abstrak di digilib
print('PROSES AMBIL LINK DIGILIB')
for x in range(1,376):
	# time.sleep(1)
	print("page : https://digilib.uns.ac.id/dokumen/fakultas/3/Fak-MIPA/"+str(x))
	response = requests.get('https://digilib.uns.ac.id/dokumen/fakultas/3/Fak-MIPA/'+str(x))
	homepage = BeautifulSoup(response.content, 'html.parser')
	links = homepage.find_all('a')
	for link in links:
		if 'https://digilib.uns.ac.id/dokumen/detail' in link.get('href'):
			link_abstraks += [link.get('href')]
with open('datanim.json', 'w') as outfile:
	outfile.write(json.dumps(link_abstraks, sort_keys=True, indent=4))

##ambil nama, nim dan abstrak berdasarkan prodi informatika dengan data link yang didapat
print("PROSES AMBIL DATA ABSTRAK")
i = 0
count = len(link_abstraks)
for link_abstrak in link_abstraks:
	try:
		count = count - 1
		response = requests.get(link_abstrak)
		page = BeautifulSoup(response.content, 'html.parser')
		links = page.find_all('tr')
		##ambil nim
		tdnim = links[2].find_all('td')
		nim = tdnim[2].get_text()
		print('remain'+str(count)+' '+nim)

		##Cek kalau prodi informatika bedasarkan nim M05
		if 'M05' in nim or 'M.05' in nim or 'M. 05' in nim or 'M 05' in nim:
			print('dapat' + nim)
			i = i+1
			array = {}
			##ambil abstrak
			tdabstrak = links[13].find_all('td')
			abstrak = tdabstrak[2].get_text()
			# array['abstrak'] = abstrak
			##ambil judul
			tdjudul = links[4].find_all('td')
			judul = tdjudul[2].get_text()
			# array['list'] = judul
			data_abstrak += [[judul,abstrak,str(i-1)]]
			with open('dataabstrak.json', 'w') as outfile:
				outfile.write(json.dumps(data_abstrak, sort_keys=True, indent=4))
	except Exception as e: 
		print(e)
print("SELESAI")
print("JUMLAH DATA ABSTRAK YANG DI DAPAT :"+str(i))

In [ ]:
#2 CASE FOLDING
import json
import re
import time
import sys
processed=[]
count=0
with open('dataabstrak.json') as json_file:
	articles = json.load(json_file)
for  article in articles:
	# folded=article[1].casefold()/
	answer = re.sub('[^a-z]+', ' ', article[1].casefold())
	articles[count][1] = answer
	count+=1
	# Progress Report
	print(str(count)+'/'+str(len(articles)))
	# sys.stdout.write("\033[F")
with open('casefolded.json', 'w') as outfile:
	outfile.write(json.dumps(articles, sort_keys=True, indent=4))




In [ ]:
!pip3 install sastrawi
##3 STEMMING
import json
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory

with open('casefolded.json') as json_file:
	articles = json.load(json_file)
count = 0
factory = StemmerFactory()
stemmer = factory.create_stemmer()

for article in articles:
	stemmed =stemmer.stem(article[1]) 
	print (str(count)+"/"+str(len(articles)))
	articles[count][1] = stemmed
	stemmed =stemmer.stem(article[2]) 
	print (str(count)+"/"+str(len(articles)))
	articles[count][2] = stemmed
	count += 1
with open('stemmed.json', 'w') as outfile:
	outfile.write(json.dumps(articles, sort_keys=True, indent=4))

In [ ]:
##4 FILTERING
threshold = 4
clustercount = 16


from os import system, name 
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█', printEnd = "\r"):
 	percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
 	filledLength = int(length * iteration // total)
 	bar = fill * filledLength + '-' * (length - filledLength)
 	print(f'\r{prefix} |{bar}| {percent}% {suffix}', end = printEnd)
 	if iteration == total:
 		print()
 		


import json
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary


with open('stemmed.json') as json_file:
	articles = json.load(json_file)
print('filtering')
count = 0
additionalstopwords=[]
# additionalstopwords+=["satu","dua","tiga","empat","lima","enam","tujuh","delapan","sembilan","sepuluh","belas","puluh","ratus","ribu","juta"]
# additionalstopwords+=["januari","februari","maret","april","mei","juni","juli","agustus","september","oktober","november","desember"]
stopwords = StopWordRemoverFactory().get_stop_words()+additionalstopwords
l = len(articles)
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
for article in articles:
	words = article[1].split()
	filtered = ""
	for word in words:
		if word not in stopwords:
			filtered += word+" " 
	articles[count][1] = filtered
	count += 1
	printProgressBar(count, l, prefix = 'Progress:', suffix = 'Complete', length = 50)


In [ ]:
!pip3 install nltk
##5 TOKENIZING

with open('filtered.json', 'w') as outfile:
	outfile.write(json.dumps(articles, sort_keys=True, indent=4))
try:
	import json
	import re
	import time
	import sys
	import nltk
except ImportError:
    pass
nltk.download('punkt')
print('tokenizing')
count=0
with open('filtered.json') as json_file:
	articles = json.load(json_file)
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
for article in articles:
	frequency = nltk.FreqDist(nltk.word_tokenize(article[1]))
	articles[count][1] = frequency
	count+=1
	printProgressBar(count, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
	# print(str(count)+'/'+str(len(articles)))
	# frequencies=nltk.FreqDist(processed)
	
# print(frequencies.most_common())
# frequencies.plot(30,cumulative=False)
# print(processed)


In [ ]:
##6 TERMFREQUENCY
with open('tokenized.json', 'w') as outfile:
	outfile.write(json.dumps(articles, sort_keys=True, indent=4))
try:
	import json
	import nltk
except ImportError:
    pass
print('Calculating Term Frequency')
count=0
counter=0
results=[]
with open('tokenized.json') as json_file:
	articles = json.load(json_file)
with open('filtered.json') as json_file:
	filtered = json.load(json_file)
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
for article in articles:
	result = {}
	# print(str(counter)+" / "+str(len(articles)))
	bagOfWordsCount = len(filtered[counter][1])
	for word, count in article[1].items():
		if((count/float(bagOfWordsCount))):
			result[word] = count / float(bagOfWordsCount)
		# print(word+str(result[word]))
	articles[counter][1]=result
	counter+=1
	printProgressBar(counter, l, prefix = 'Progress:', suffix = 'Complete', length = 50)


In [ ]:
##7 INVERSE DOCUMENT FREQUENCY
with open('termfrequency.json','w') as outfile:
	outfile.write(json.dumps(articles, sort_keys=True, indent=4))	

try:
	import json
	import nltk
	import math
except ImportError:
    pass
count=0
counter=0
with open('tokenized.json') as json_file:
	articles = json.load(json_file)
print('Calculating Inverse Document Frequency')
N = len(articles)
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
for article in articles:
	idfDict = dict.fromkeys(article[1].keys(), 0)
	result = {}
	for key in idfDict.keys():
		for article in articles:
			if key in article[1]:
				idfDict[key] += 1
	for word, val in idfDict.items():
		# if 50>val>4:
		if val>threshold:
			result[word] = math.log10(N / (float(val)+1))
		# else:
		# 	print(word+" omitted, df = "+str(val))

	articles[count][1] = result
	count+=1
	printProgressBar(count, l, prefix = 'Progress:', suffix = 'Complete', length = 50)



In [ ]:
##8 TFIDF
with open('idfed.json','w') as outfile:
	outfile.write(json.dumps(articles, sort_keys=True, indent=4))	


try:
	import json
except ImportError:
    pass
print('Calculating TF-IDF')
count=0
counter=0
tfidf = []
with open('idfed.json') as json_file:
	idfs = json.load(json_file)
with open('termfrequency.json') as json_file:
	tfs = json.load(json_file)
printProgressBar(0, l, prefix = 'Progress:', suffix = 'Complete', length = 50)
for idf in idfs:
	tfidf={}
	tfs_dict = tfs[count][1].items()
	maxtfidf = 0
	for word, val in idf[1].items():
		# if val*tfs[count][1].get(word)>0:
			# print(val*tfs[count][1].get(word))
		if tfs[count][1].get(word) is not None:
			tfidf[word] = val*float(tfs[count][1].get(word))
			if maxtfidf<tfidf[word]:
				maxtfidf=tfidf[word]
				maxtfidfword = word
	idfs[count][1]=tfidf
	idfs[count] += [maxtfidfword]

	count+=1
	printProgressBar(count, l, prefix = 'Progress:', suffix = 'Complete', length = 50)




In [ ]:
#9 DICTIONARY
with open('tfidfed.json','w') as outfile:
	outfile.write(json.dumps(idfs, sort_keys=True, indent=4))	
try:
	import json
except ImportError:
    pass
print('Building dictionary')

count1=0

i=0
tfidf = []
words={}
result=[]
with open('tfidfed.json') as json_file:
	documents = json.load(json_file)
# print (documents[0][1])
# print (documents[0].pop(1))
for document in documents:
	count2=0
	cek={}
	for word in document[1].items():
		# result.append(count1)
		if word[0] in words.values():
			# print("match")
			for key,value in words.items():
				if word[0] == value:
					# print("match "+word[0])
					cek[key]=word[1]
					# print(word[1])
		else:
			# print("added "+str(i)+" = "+word[0])
			words[i]=word[0]
			# print(words[i]+" = "+str(word[0]))
			cek[i]=word[1]
			i+=1;
			# print('added '+word)
		count2+=1

	# print(cek)

	result=[cek]
	# documents[count1].pop(1)
	documents[count1][1]=cek

	count1+=1

# print(words)
# print(words)

# print(count1)

with open('dictionary.json','w') as outfile:
	outfile.write(json.dumps(words, sort_keys=True, indent=4))	

with open('dictionarized.json','w') as outfile:
	outfile.write(json.dumps(documents, sort_keys=True, indent=4))	


In [ ]:
#10 hierarchical clustering
import json
import numpy as np
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
import scipy.cluster.vq as scv
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from pandas import DataFrame
from sklearn.neighbors import NearestCentroid
from sklearn.manifold import MDS

with open('dictionarized.json') as json_file:
	documents = json.load(json_file)
with open('dictionary.json') as json_file:
	dictionary = json.load(json_file)
count=0

# for document in documents:
# 	documents[count][0]=count
# 	count+=1
# X = [[i] for i in documents]
# print(X)
print('Calculating Clusters')
array=[]

for document in documents:
	count2=0
	array.append([])
	for key in dictionary.keys():
		
		if not document[1].get(key) is None:
			array[count].append(document[1].get(key))
		else:
			array[count].append(0)
	count+=1
	# if count == 10:
	# 	break;
X=np.array(array)
counter=0
titles=[]
kanal = []
subkanal = []
topword = []
for n in documents:
	titles+=[n[0]]
	# kanal+=[n[2]]
	# subkanal+=[n[3]]
	# topword+=[n[4]]
	counter+=1


# print(X)
# X.set_index(titles)
# print(X)

clustering = AgglomerativeClustering(n_clusters=clustercount, affinity='euclidean', linkage='ward')
cluster = clustering.fit_predict(X)
print(clustering.labels_)
# plt.scatter(X[:,0],X[:,1], c=cluster.labels_, cmap='rainbow')
# plt.show()

clf = NearestCentroid()
clf.fit(X, cluster)
centroids = clf.centroids_
# print(array[200])
fig, dendogram = plt.subplots(figsize=(30, 40))
dendrogram = sch.dendrogram(sch.linkage(array, method  = "ward"),orientation = "right")
plt.title('Dendrogram')
plt.ylabel('Article')
plt.xlabel('Euclidean distances')
plt.tick_params(\
    axis= 'x',          # changes apply to the x-axis
    which='both',      # both major and minor ticks are affected
    bottom='off',      # ticks along the bottom edge are off
    top='off',         # ticks along the top edge are off
    labelbottom='off')
plt.tight_layout()

# plt.show()
plt.savefig('t'+str(threshold)+'sw'+str(len(additionalstopwords))+'dendogram.png', dpi=200)




df = DataFrame(array,columns=dictionary.values())
  


In [ ]:
#11 K-MEANS
kmeans = KMeans(n_clusters=clustercount, init=centroids, n_init=1, max_iter=1000).fit(df)
centroids = kmeans.cluster_centers_
print(kmeans.labels_)
# print(df)
clusters=kmeans.labels_.tolist()
# print(clusters)
print("threshold = "+str(threshold))
print("thresholdmax = "+str(thresholdmax))
print("stopwords = "+str(additionalstopwords))
for n in range(clustercount):
	print("cluster "+str(n)+" = "+str(clusters.count(n)))
dataframe = DataFrame({
	"count":range(len(titles)),
	"cluster":clusters,
	"title":titles,
	# "kanal":kanal,
	# "subkanal":subkanal,
	# "top word":topword

	})
sortedres=dataframe.sort_values('cluster')
sortedres.to_csv('t'+str(threshold)+'sw'+str(len(additionalstopwords))+'result.csv')
# with open('result.json','w') as outfile:
	# outfile.write(json.dumps(dataframe, sort_keys=True, indent=4))	
with open('t'+str(threshold)+'sw'+str(len(additionalstopwords))+'summary.txt', 'w') as f:
	print("threshold = "+str(threshold), file=f)
	print("stopwords = "+str(additionalstopwords), file=f)
	for n in range(10):
		print("cluster "+str(n)+" = "+str(clusters.count(n)), file=f)




with open('filtered.json') as json_file:
	documents = json.load(json_file)
clusterwords = [""]*clustercount
count = 0
for cluster in clusters:
	clusterwords[cluster]+=documents[count][1]
	count+=1
count = 0
for clusterword in clusterwords:
	frequency = nltk.FreqDist(nltk.word_tokenize(clusterword))
	clusterwords[count] = frequency
	count+=1
	# frequencies=nl

from collections import Counter
count = 0
for clusterword in clusterwords:
	
	clusterwords[count]=dict(Counter(clusterword).most_common(20))
	count+=1
with open('t'+str(threshold)+'sw'+str(len(additionalstopwords))+'clusterword.json','w') as outfile:
	outfile.write(json.dumps(clusterwords, sort_keys=True, indent=4))	